# Job 9 - CAS RÉEL : Émettre des factures

## Mise en situation

Une petite boutique veut générer des factures chaque mois.

Elle a:
- Une table clients
- Une table commandes
- Une table articles

Tâche: Créer les requêtes qui permettent de générer une facture pour le client X.

**Durée** : ~10 min  
**Prérequis** : Jobs 1-8

In [ ]:
import sqlite3
import pandas as pd
from datetime import datetime, timedelta

conn = sqlite3.connect(':memory:')
cursor = conn.cursor()

# Créer les tables
cursor.execute('''
CREATE TABLE clients (
    id INTEGER PRIMARY KEY,
    nom TEXT,
    adresse TEXT,
    email TEXT
)
''')

cursor.execute('''
CREATE TABLE articles (
    id INTEGER PRIMARY KEY,
    nom TEXT,
    prix REAL
)
''')

cursor.execute('''
CREATE TABLE commandes (
    id INTEGER PRIMARY KEY,
    client_id INTEGER,
    date_commande DATE,
    statut TEXT
)
''')

cursor.execute('''
CREATE TABLE commande_articles (
    id INTEGER PRIMARY KEY,
    commande_id INTEGER,
    article_id INTEGER,
    quantite INTEGER
)
''')

# Insérer des données
cursor.executemany('INSERT INTO clients VALUES (?, ?, ?, ?)', [
    (1, 'Alice Martin', '123 Rue de Paris', 'alice@example.com'),
    (2, 'Bob Dupont', '456 Rue de Lyon', 'bob@example.com')
])

cursor.executemany('INSERT INTO articles VALUES (?, ?, ?)', [
    (1, 'Laptop', 999.99),
    (2, 'Souris', 29.99),
    (3, 'Clavier', 149.99)
])

cursor.executemany('INSERT INTO commandes VALUES (?, ?, ?, ?)', [
    (1, 1, '2024-01-15', 'livree'),
    (2, 1, '2024-02-10', 'en_cours'),
    (3, 2, '2024-01-20', 'livree')
])

cursor.executemany('INSERT INTO commande_articles VALUES (?, ?, ?, ?)', [
    (1, 1, 1, 1),
    (2, 1, 2, 2),
    (3, 2, 3, 1),
    (4, 3, 1, 1),
    (5, 3, 3, 1)
])

conn.commit()
print('Boutique simulée créée')

## Étape 1 : Générer la facture du client 1

Récupère: Client, ses commandes, les articles dans chaque commande

In [ ]:
client_id = 1

# Récupérer les infos du client
cursor.execute('SELECT * FROM clients WHERE id = ?', (client_id,))
client = cursor.fetchone()
print(f'Facture pour: {client[1]}')
print(f'Adresse: {client[2]}')
print('\n' + '='*50 + '\n')

## Étape 2 : Affiche chaque commande avec ses articles

In [ ]:
cursor.execute('''
SELECT c.id, c.date_commande, a.nom, a.prix, ca.quantite, (a.prix * ca.quantite) as total
FROM commandes c
JOIN commande_articles ca ON c.id = ca.commande_id
JOIN articles a ON ca.article_id = a.id
WHERE c.client_id = ?
ORDER BY c.id
''', (client_id,))

resultats = cursor.fetchall()
df = pd.DataFrame(resultats, columns=['Commande', 'Date', 'Article', 'Prix', 'Quantité', 'Total'])
print('Détail des commandes:')
print(df)

## Étape 3 : Montant total du client

In [ ]:
cursor.execute('''
SELECT SUM(a.prix * ca.quantite) as total
FROM commandes c
JOIN commande_articles ca ON c.id = ca.commande_id
JOIN articles a ON ca.article_id = a.id
WHERE c.client_id = ?
''', (client_id,))

montant_total = cursor.fetchone()[0]
print(f'\nMONTANT TOTAL: {montant_total:.2f} EUR')

## Étape 4 : Le client qui a dépensé le plus

In [ ]:
cursor.execute('''
SELECT c.nom, COUNT(cmd.id) as nb_commandes, SUM(a.prix * ca.quantite) as total_depense
FROM clients c
LEFT JOIN commandes cmd ON c.id = cmd.client_id
LEFT JOIN commande_articles ca ON cmd.id = ca.commande_id
LEFT JOIN articles a ON ca.article_id = a.id
GROUP BY c.id
ORDER BY total_depense DESC
'''
)

resultats = cursor.fetchall()
df = pd.DataFrame(resultats, columns=['Client', 'Commandes', 'Total dépensé'])
print('\nRanking des clients:')
print(df)

## À toi

1. Qui a commandé le Laptop ?
2. Quel est l'article le plus cher commandé ?
3. Combien d'articles différents chaque client a commandé ?
4. Ajoute une nouvelle commande et refais les requêtes